# Text generation

[Live-demo by HuggingFace](https://transformer.huggingface.co/doc/gpt2-large)

In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
MODEL_NAME = "gpt2"

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [4]:
text = "My favourite movie is"

In [5]:
tokenizer.tokenize(text)

['My', 'Ġfavourite', 'Ġmovie', 'Ġis']

In [6]:
text_encoded = torch.tensor([tokenizer.encode(text)], dtype=torch.long)

In [7]:
text_encoded

tensor([[ 3666, 12507,  3807,   318]])

In [8]:
with torch.no_grad():
    predictions = model(text_encoded)

In [9]:
predictions[0].shape

torch.Size([1, 4, 50257])

In [10]:
next_token_logits = predictions[0][:, -1, :]

In [11]:
next_token_logits.shape

torch.Size([1, 50257])

In [12]:
next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

In [13]:
tokenizer.convert_ids_to_tokens(next_token)

['ĠThe']

In [14]:
def generate_text(input_text: str, tokens_to_generate: int):
    text_generated = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
    result = text
    with torch.no_grad():
        for _ in range(tokens_to_generate):
            predictions = model(text_generated)
            next_token_logits = predictions[0][:, -1, :]

            next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)
            
            text_generated = torch.cat((text_generated, next_token_id), dim=1)
            result = tokenizer.decode(text_generated.squeeze().tolist())
    return result

In [20]:
print(generate_text(text, 15))

My favourite movie is The Matrix, and I'm not sure if I'll ever watch it again


In [19]:
print(generate_text(text, 50))

My favourite movie is The Matrix, and I'm not sure if I'll ever watch it again.

I'm not sure if I'll ever watch it again. I'm not sure if I'll ever watch it again. I'm not sure if I'll ever


## Улучшение подходов генерации текста

![image](https://miro.medium.com/max/1400/1*9sEpLZF8lV5OXwUQUMVZlg.png)

### Сэмплирование с температурой

Сэмплирование с температурой основана на статистической термодинамике, где высокая температура означает, что с большей вероятностью встречаются состояния с низкой энергией. В вероятностных моделях логиты играют роль энергии, и мы можем реализовать сэмплирование с температурой, разделив логиты на температуру перед подачей их в softmax и получением сэмплированных вероятностей.

Более низкие температуры делают модель более уверенной в выборе, в то время как температуры выше 1 снижают уверенность. Температура 0 эквивалентна правдоподобию argmax/max, а бесконечная температура соответствует равномерной выборке.

### Top K sampling

Top-k сэмплирование означает сортировку по вероятности и обнуление вероятностей для всего, что ниже k-го токена. На практике получается так, что это улучшает качество за счет удаления хвоста и уменьшения вероятности отклонения от темы. Но в некоторых случаях действительно есть много слов, из которых мы могли бы разумно выбрать (широкое распределение ниже), а в некоторых случаях нет (узкое распределение ниже).

![image](https://miro.medium.com/max/1400/0*J37qonVPJvKZpzv2)


### Top P сэмплирование (nuclear sampling)


Чтобы решить эту проблему, предлагаются использовать top p сэмплирование, также известную как nucleus sampling, в которой мы вычисляем кумулятивное распределение и отсекаем, как только [CDF](https://en.wikipedia.org/wiki/Cumulative_distribution_function) достигает P. В приведенном выше примере с широким распределением для достижения top_p = 0.9 может потребоваться больше 100 токенов. В узком распределении мы можем уже превысить top_p = 0.9 только с «hot» и «warm» в нашей выборке. Таким образом, мы по-прежнему избегаем выборки явно неправильных токенов, но сохраняем разнообразие, когда токены с наивысшей оценкой имеют слабую уверенность.
Почему не работает выборка методом максимального правдоподобия? В процессе обучения никогда не бывает сложных ошибок. Модель обучена предсказывать следующий токен на основе контекста, созданного человеком. Если она ошибается с одним токеном из-за неправильного распределения, следующий токен использует «правильный» контекст, созданный человеком, независимо от последнего прогноза. Во время генерации он вынужден завершить свой собственный автоматически сгенерированный контекст, параметр, который он не учел во время обучения.

[Статья с детальным описанием](https://arxiv.org/pdf/1904.09751.pdf)